# Problem Description:

* Need to save data into a Relationa Database.
* The problem was that data does not have same columns for all sources. 
* Originally the data was coming as a JSON file, and was imported as a Pandas Dataframe.
* To simulate this problem I will be using different csv files where data was generated to keep original data confidentialy.
* All data columns are strings




# Solution:


* Create a function that will generate dynamically an INSERT query according to columns readed from the input file.



In [1]:
import pandas as pd
import numpy as np
import json
import os

import mysql.connector
from mysql.connector import Error

In [2]:
# Get  login and password from environment variables
USER = os.getenv('MYSQL_USER')
PASSWORD = os.getenv('MYSQL_PASSWORD')


In [3]:
df = pd.read_csv("./file1.csv", dtype=str)

# Some databases do not allow integer as column names.
# Add "col_" prefix to columns
df = df.add_prefix("col_") 
df.head()

,col_t,col_p,col_1,col_2,col_3,col_4,col_7,col_11
0,0430,F2C30DC8000,00000000001,100010004305,004300,034552240,0435463,40Y4025


## Functions:

In [9]:
# Function to connect to MySQL and Insert Data:
def database_insert(insert_query, vals):
    try:
        connection = mysql.connector.connect(host='localhost',
                                         database='testedb',
                                         user=USER,
                                         password=PASSWORD)
    
        cursor = connection.cursor()
        result = cursor.execute(insert_query, vals)
        connection.commit()
        print("Inserted successfully ")

    except mysql.connector.Error as error:
        print("Failed to insert in MySQL: {}".format(error))
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

# Function to read CSV files and return the "Insert Query" used to insert data to MySQL
def processa_arquivo(filename):
    df = pd.read_csv(filename, dtype=str)
    df = df.add_prefix("col_")
    
    # Create a list of columns that will be used on insert
    colunas, valores = df.columns, df.values
    cols = cols = colunas.tolist()
    vals = valores[0].tolist()
    
    # Generate query to insert into columns dynamically according to columns from input file.
    query_placeholders = ', '.join(['%s'] * len(vals))
    query_columns = ', '.join(cols)
    insert_query = ''' INSERT INTO OPERADORAS (%s) VALUES (%s) ''' %(query_columns, query_placeholders)
    
    # Perform insert into database
    database_insert(insert_query, vals)

# Function to perform Select on MySQL to validate data:
def consulta_db():
    try:
        connection = mysql.connector.connect(host='localhost',
                                             database='testedb',
                                             user=USER,
                                             password=PASSWORD)

        cursor = connection.cursor()
        select_query = "SELECT * FROM OPERADORAS"
        result = cursor.execute(select_query)
        resultado = cursor.fetchall()

    except mysql.connector.Error as error:
        print("Failed to insert in MySQL: {}".format(error))
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            #print("MySQL connection is closed")
    return(resultado)


## Create Table - MySQL Database

In [23]:
# Create table at MySQL Database
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='testedb',
                                         user=USER,
                                         password=PASSWORD)
    
    #mySql_Drop_Table_Query = """DROP TABLE OPERADORAS"""
    mySql_Create_Table_Query = """CREATE TABLE OPERADORAS ( 
                             Id int(11) NOT NULL AUTO_INCREMENT,
                             col_t varchar(250),
                             col_p varchar(250),
                             col_1 varchar(250),
                             col_2 varchar(250),
                             col_3 varchar(250),
                             col_4 varchar(250),
                             col_5 varchar(250),
                             col_6 varchar(250),
                             col_7 varchar(250),
                             col_8 varchar(250),
                             col_9 varchar(250),
                             col_10 varchar(250),
                             col_11 varchar(250),
                             col_12 varchar(250),
                             col_13 varchar(250),
                             col_14 varchar(250),
                             col_15 varchar(250),
                             col_16 varchar(250),
                             col_17 varchar(250),
                             col_18 varchar(250),
                             col_19 varchar(250),
                             col_20 varchar(250),
                             col_21 varchar(250),
                             col_22 varchar(250),
                             col_32 varchar(250),
                             col_39 varchar(250),
                             col_41 varchar(250),
                             col_42 varchar(250),
                             col_49 varchar(250),
                             col_127 varchar(250),
                             col_operadora varchar(250),
                             PRIMARY KEY (Id)) """
                             
    cursor = connection.cursor()
    #result = cursor.execute(mySql_Drop_Table_Query)
    result = cursor.execute(mySql_Create_Table_Query)
    print("Table created successfully ")

except mysql.connector.Error as error:
    print("Failed to create table in MySQL: {}".format(error))
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Failed to create table in MySQL: 1050 (42S01): Table 'OPERADORAS' already exists
MySQL connection is closed


## Processing File:

In [21]:
# Process file and insert into database
processa_arquivo('./file2.csv')
#print(insert)

Inserted successfully 
MySQL connection is closed


## Check if data was inserted at Database

In [25]:
# Consultar banco de dados:
resultado = consulta_db()

for res in resultado:
    print(res)

(1, '0202', 'F23C040102C08000', '0000000000000002', '1001063405633211', '003000', '000000005351', None, None, '0621133311', None, None, None, '210201', '103251', '0621', '3001', None, None, None, None, None, None, None, '021', '00000000634', '00', 'KW021209', '000000000006228', '986', '003471201', 'PrivateLabel_2')
(2, '0202', 'F23C040102C08000', '0000000000000002', '1001063405633211', '003000', '000000005351', None, None, '0621133311', None, None, None, '210201', '103251', '0621', '3001', None, None, None, None, None, None, None, '021', '00000000634', '00', 'KW021209', '000000000006228', '986', '003471201', 'PrivateLabel_2')
(3, '0430', 'F2C30DC8000', '00000000001', '100010004305', '004300', '034552240', None, None, '0435463', None, None, None, '40Y4025', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
(4, '0430', 'F2C30DC8000', '00000000001', '100010004305', '004300', '034552240', None, None, '0435463', None, None, None, '40